## Notebook pour la récupération des commentaires des utilisateurs sur IMBD via Selenium

In [9]:
#pip install selenium

In [1]:
import time
import requests
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
data_bo = pd.read_csv("data_bo.csv")
data_bo.head()

,Titre,Réalisateur,Année,Pays De Production,Recettes,Meilleur Rang
0,Avatar,James Cameron,2009,États-Unis,2923706026,1
1,Avengers: Endgame,Anthony etJoe Russo,2019,États-Unis,2799439100,1
2,Avatar : La Voie de l'eau,James Cameron,2022,États-Unis,2320250281,3
3,Titanic,James Cameron,1997,États-Unis,2264743305,1
4,"Star Wars, épisode VII : Le Réveil de la Force",J. J. Abrams,2015,États-Unis,2071310218,3


In [53]:
data_bo['Titre']

0                                                Avatar
1                                     Avengers: Endgame
2                             Avatar : La Voie de l'eau
3                                               Titanic
4        Star Wars, épisode VII : Le Réveil de la Force
5                                Avengers: Infinity War
6                               Spider-Man: No Way Home
7                                        Jurassic World
8                                           Le Roi lion
9                                              Avengers
10                                   Fast and Furious 7
11                                   Top Gun : Maverick
12                                La Reine des neiges 2
13                                               Barbie
14                            Avengers : L'Ère d'Ultron
15                            Super Mario Bros. le film
16                                        Black Panther
17    Harry Potter et les Reliques de la Mort : 

!!! attention !!! Il faut télécharger un "chromedriver" de la même version que votre version chrome, et le mettre à coté de ce fichier jupyter dans les fichiers.

In [3]:
def scrape_imdb_reviews(url, num_iterations=3, wait_time=20):
    # Initialise le navigateur
    driver = webdriver.Chrome()
    driver.get(url)

    wait = WebDriverWait(driver, wait_time)
    
    # Locate the span element with the specified class
    span_element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "sc-bde20123-1")))

    # Get the text content of the span element
    note = span_element.text

    #  Print or use the extracted text
    print(note)
    
    time.sleep(4)

    # Get the current scroll position
    current_scroll_position = driver.execute_script("return window.scrollY;")

    # Scroll down by 200 pixels from the current position
    scroll_by_pixels = 400
    new_scroll_position = current_scroll_position + scroll_by_pixels
    driver.execute_script(f"window.scrollTo(0, {new_scroll_position});")
    
    time.sleep(2)
    
    user_comments_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.three-Elements > span.label")))

    # Cliquez sur le bouton
    user_comments_button.click()
    wait = WebDriverWait(driver, wait_time)
    
    
    #cherche le bouton "Hide Spoilers"
    spoiler_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "faceter-facets-text")))
    # Cliquez sur le bouton
    spoiler_button.click()
    
    # Clique sur le bouton "Load More" plusieurs fois
    for i in range(num_iterations):
        load_more_button = wait.until(EC.element_to_be_clickable((By.ID, "load-more-trigger")))
        load_more_button.click()
        driver.implicitly_wait(3)

    time.sleep(3)

    # Trouve les commentaires par classe CSS
    reviews = driver.find_elements(By.CSS_SELECTOR, ".text.show-more__control")

    # Imprime les commentaires pour débugger (les trois premiers commentaires)
    
    reviews_txt = []
    for review in reviews:
        reviews_txt.append(review.text) 

    # Imprime le nombre total de commentaires récupérés
    print(f"Nombre total de commentaires : {len(reviews)}")

    time.sleep(2)

    # Ferme le navigateur
    driver.quit()
    
    return reviews_txt, note

Les commentaires cachés avec "warning spoil" sont maintenant vides dans notre liste, contrairement à avant avec la méthode beautiful soupe.

Le lien "hidespoil"ci-dessous est ce lien:https://www.imdb.com/title/tt0499549/reviews?spoiler=hide&sort=curated&dir=desc&ratingFilter=0
où l'on a coché la case "hide spoiler" ce qui retire automatiquement de la liste les commentaires avec la banière warning spoil. Ce qui nous permet de ne pas avoir d'élément vide dans notre liste. 
J'ai donc modifié la fonction pour que le robot clique automatiquement sur le bouton "hide spoilers" ce qui permet d'aller vers le 2ème lien à partir du 1er lien automatiquement.

In [59]:
url_avatar = "https://www.imdb.com/title/tt0499549/"
reviews_avatar = scrape_imdb_reviews(url_avatar, num_iterations=3, wait_time=20)

7,9
["I was waiting for this day for the longest time. I was a kid back in 2009 when this movie released. So never got to watch it back then. But now when it rereleased I booked the first day show to a IMAX show and oh boy was I blown away! This is nothing short of a masterpiece! It's beyond belief how a film like this could've been made. Every scene, every shot is perfection. You are transferred to a different world and become so engrossed in the film. Never have I attended a movie where at the end of it people got up from their seats and started clapping! Last time this happened was after Infinity War. That movie too was a damn brilliant one. I'm from India and this is legit rare, where people go crazy, so crazy after any English film. This was one experience that I will never forget. I'm definitely going for it again next weekend cuz just once was not enough.\n\nEvery human on the planet needs to experience this magnificent work of art!", "Forgive me, I'm going to jump from professi

Les commentaires cachés avec "warning spoil" sont maintenant vides dans notre liste, contrairement à avant avec la méthode beautiful soupe.

In [4]:
#vérification que nous avons pas de commentaire "vide"
X = np.array([len(review) for review in reviews_avatar_2])
len(X)
X

NameError: name 'reviews_avatar_2' is not defined

In [61]:
#Test pour un autre film
url_cars = "https://www.imdb.com/title/tt0317219/?ref_=nv_sr_srsg_0_tt_8_nm_0_q_cars"
reviews_cars = scrape_imdb_reviews(url_cars, num_iterations=3, wait_time=20)

7,2
Nombre total de commentaires : 100


In [40]:
def scrap_review_all(movie_titles, release_year, wait_time=10, num_iterations=3, browser="chrome"):
    if browser == "chrome":
        driver = webdriver.Chrome()
    else:
        # Add support for other browsers if needed
        raise ValueError("Unsupported browser")
    
    dico_all_comments = {}
    driver.get("https://www.google.com")
    wait = WebDriverWait(driver, wait_time)
    # Attendre que le bouton soit présent
    button = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='QS5gu sy4vM' and text()='Tout refuser']")))
        
    # Cliquez sur le bouton
    button.click()
    
    for i in range(len(movie_titles)):
        # Ouvrir Google
        
        cleaned_title = movie_titles[i].strip().replace("\xa0", " ")
        driver.get("https://www.google.com")
        wait = WebDriverWait(driver, wait_time)
        
        
        
        search_box = driver.find_element(By.NAME, "q")
        search_box.send_keys("imdb title " + cleaned_title + " " + str(release_year[i]))
        print("imdb title " + cleaned_title + " " + str(release_year[i]))
        search_box.send_keys(Keys.RETURN)
        
        # Attente que les résultats de recherche soient chargés
        wait.until(EC.presence_of_element_located((By.ID, "search")))
        
        time.sleep(1.5)
        
        # Trouver le premier lien dans les résultats de recherche et cliquer dessus
        first_link = driver.find_element(By.CSS_SELECTOR, "h3")
        first_link.click()
        
        time.sleep(1)
        
        # Locate the span element with the specified class
        span_element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "sc-bde20123-1")))

        # Get the text content of the span element
        note = span_element.text

        #  Print or use the extracted text
        print(note)
    
        time.sleep(1.5)

        # Get the current scroll position
        current_scroll_position = driver.execute_script("return window.scrollY;")

           # Scroll down by 200 pixels from the current position
        scroll_by_pixels = 400
        new_scroll_position = current_scroll_position + scroll_by_pixels
        driver.execute_script(f"window.scrollTo(0, {new_scroll_position});")
    
        time.sleep(2)
    
        user_comments_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.three-Elements > span.label")))

         # Cliquez sur le bouton
        user_comments_button.click()
        wait = WebDriverWait(driver, wait_time)
        
        time.sleep(1)
    
    
         #cherche le bouton "Hide Spoilers"
        spoiler_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "faceter-facets-text")))
        # Cliquez sur le bouton
        spoiler_button.click()
        
 
        # Clique sur le bouton "Load More" plusieurs fois
        for i in range(num_iterations):
            load_more_button = wait.until(EC.element_to_be_clickable((By.ID, "load-more-trigger")))
            load_more_button.click()
            time.sleep(2)

        time.sleep(3)

        # Trouve les commentaires par classe CSS
        reviews = driver.find_elements(By.CSS_SELECTOR, ".text.show-more__control")

        print( cleaned_title)
        reviews_txt = [review.text for review in reviews]
        dico_all_comments[cleaned_title] = [reviews_txt, note]
        time.sleep(3)
    
    driver.quit()
    return dico_all_comments

In [41]:
comments_all_1 = scrap_review_all(list(data_bo.iloc[:25,0]), list(data_bo.iloc[:25,2]), wait_time=20)

imdb title Avatar 2009
7,9
Avatar
imdb title Avengers: Endgame 2019
8,4
Avengers: Endgame
imdb title Avatar : La Voie de l'eau 2022
7,6
Avatar : La Voie de l'eau
imdb title Titanic 1997
7,9
Titanic
imdb title Star Wars, épisode VII : Le Réveil de la Force 2015
7,8
Star Wars, épisode VII : Le Réveil de la Force
imdb title Avengers: Infinity War 2018
8,4
Avengers: Infinity War
imdb title Spider-Man: No Way Home 2021
8,2
Spider-Man: No Way Home
imdb title Jurassic World 2015
6,9
Jurassic World
imdb title Le Roi lion 2019
6,8
Le Roi lion
imdb title Avengers 2012
8,0
Avengers
imdb title Fast and Furious 7 2015
7,1
Fast and Furious 7
imdb title Top Gun : Maverick 2022
8,3
Top Gun : Maverick
imdb title La Reine des neiges 2 2019
6,8
La Reine des neiges 2
imdb title Barbie 2023
7,0
Barbie
imdb title Avengers : L'Ère d'Ultron 2015
7,3
Avengers : L'Ère d'Ultron
imdb title Super Mario Bros. le film 2023
7,1
Super Mario Bros. le film
imdb title Black Panther 2018
7,3
Black Panther
imdb title Harry

In [45]:
print(comments_all_1.keys())
print(len(comments_all_1.keys()))
for i in comments_all_1.keys():
    print( len(comments_all_1[i][0]))

dict_keys(['Avatar', 'Avengers: Endgame', "Avatar : La Voie de l'eau", 'Titanic', 'Star Wars, épisode VII : Le Réveil de la Force', 'Avengers: Infinity War', 'Spider-Man: No Way Home', 'Jurassic World', 'Le Roi lion', 'Avengers', 'Fast and Furious 7', 'Top Gun : Maverick', 'La Reine des neiges 2', 'Barbie', "Avengers : L'Ère d'Ultron", 'Super Mario Bros. le film', 'Black Panther', 'Harry Potter et les Reliques de la Mort : Partie 2', 'Star Wars, épisode VIII : Les Derniers Jedi', 'La Reine des neiges', 'Jurassic World: Fallen Kingdom', 'La Belle et la Bête', 'Les Indestructibles 2', 'Fast and Furious 8', 'Iron Man 3'])
25


In [52]:
comments_all_1['Les Indestructibles 2'][0][:2]

['While I understand the disappointment, this was NOT a bad film. It isn\'t as good as the first one, but I think it holds up pretty well! There\'s lots of fun to be had watching this from the animation to the humor, and there are still some touching scenes that show there is heart put into this film. I recommend seeing CinemaWins "Everything GREAT about Incredibles 2" if you want to see why this film is still pretty decent. Only somewhat weak spot in my opinion was the main villain who was somewhat predictable, but man the animation at times truly is "Incredible" (pun intended)!\n\nGo see it! It was well worth my time and I enjoyed it, so I\'m surprised to see it get all these 1 star reviews. Far from Pixar\'s best but still a good flick. I know you can\'t please everyone but I think this movie is terribliy overhated. That\'s just my opinion at least. 9 stars out of 10!',
 "Good: The animation for this movie was excellent just like most other Pixar movies, it was funny (especially Jac

In [79]:
comments_all_2 = scrap_review_all(list(data_bo.iloc[25:40,0]), list(data_bo.iloc[25:40,2]), wait_time=20)

imdb title Les Minions 2015
6,4
Les Minions
imdb title Le Seigneur des anneaux : Le Retour du roi 2003
9,0
Le Seigneur des anneaux : Le Retour du roi
imdb title Captain America: Civil War 2016
7,8
Captain America: Civil War
imdb title Aquaman 2018
6,8
Aquaman
imdb title Skyfall 2012
7,8
Skyfall
imdb title Spider-Man: Far From Home 2019
7,4
Spider-Man: Far From Home
imdb title Captain Marvel 2019
6,8
Captain Marvel
imdb title Transformers 3 : La Face cachée de la Lune 2011
6,2
Transformers 3 : La Face cachée de la Lune
imdb title Jurassic Park 1993
8,2
Jurassic Park
imdb title Transformers : L'Âge de l'extinction 2014
5,6
Transformers : L'Âge de l'extinction
imdb title The Dark Knight Rises 2012
8,4
The Dark Knight Rises
imdb title Joker 2019
8,4
Joker
imdb title Star Wars, épisode IX : L'Ascension de Skywalker 2019
6,4
Star Wars, épisode IX : L'Ascension de Skywalker
imdb title Toy Story 4 2019
7,7
Toy Story 4
imdb title Toy Story 3 2010
8,3
Toy Story 3


In [81]:
print(comments_all_2.keys())
print(len(comments_all_2.keys()))
for i in comments_all_2.keys():
    print( len(comments_all_2[i][0]))

dict_keys(['Les Minions', 'Le Seigneur des anneaux : Le Retour du roi', 'Captain America: Civil War', 'Aquaman', 'Skyfall', 'Spider-Man: Far From Home', 'Captain Marvel', 'Transformers 3 : La Face cachée de la Lune', 'Jurassic Park', "Transformers : L'Âge de l'extinction", 'The Dark Knight Rises', 'Joker', "Star Wars, épisode IX : L'Ascension de Skywalker", 'Toy Story 4', 'Toy Story 3'])
15
100
100
100
100
99
100
100
100
100
100
100
99
100
100
100


In [65]:
comments_all_3 =  scrap_review_all(list(data_bo.iloc[41:,0]), list(data_bo.iloc[41:,2]), wait_time=20)

imdb title Rogue One: A Star Wars Story 2016
7,8
Rogue One: A Star Wars Story
imdb title Aladdin 2019
6,9
Aladdin
imdb title Pirates des Caraïbes : La Fontaine de Jouvence 2011
6,6
Pirates des Caraïbes : La Fontaine de Jouvence
imdb title Moi, moche et méchant 3 2017
6,2
Moi, moche et méchant 3
imdb title Le Monde de Dory 2016
7,2
Le Monde de Dory
imdb title Star Wars, épisode I : La Menace fantôme 1999
6,5
Star Wars, épisode I : La Menace fantôme
imdb title Zootopie 2016
8,0
Zootopie
imdb title Alice au pays des merveilles 2010
6,4
Alice au pays des merveilles
imdb title Harry Potter à l'école des sorciers 2001
7,6
Harry Potter à l'école des sorciers


In [66]:
print(comments_all_3.keys())
print(len(comments_all_3.keys()))
for i in comments_all_3.keys():
    print( len(comments_all_3[i][0]))

dict_keys(['Rogue One: A Star Wars Story', 'Aladdin', 'Pirates des Caraïbes : La Fontaine de Jouvence', 'Moi, moche et méchant 3', 'Le Monde de Dory', 'Star Wars, épisode I : La Menace fantôme', 'Zootopie', 'Alice au pays des merveilles', "Harry Potter à l'école des sorciers"])
9
100
100
100
100
100
97
100
100
100


In [82]:
pirate_caraibe_NA = {'Pirates des Caraïbes : Le Secret du coffre maudit': [np.nan, np.nan]}
pirate_caraibe_NA

{'Pirates des Caraïbes : Le Secret du coffre maudit': [nan, nan]}

In [91]:
comments_all_1.update(comments_all_2)
comments_all_1.update(pirate_caraibe_NA)
comments_all_1.update(comments_all_3)
data_comments = pd.DataFrame(comments_all_1)

In [95]:
data_comments.to_json('data_reviews')